# Handwritten Digit Classification using a Convolutional Neural Network (CNN)

This notebook walks through the process of building, training, and evaluating a CNN for classifying handwritten characters from the provided dataset.

## 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 2. Data Loading and Preprocessing

In [ ]:
# Load the CSV file
csv_path = 'dataset/english.csv'
data = pd.read_csv(csv_path)

# Display the first few rows of the dataframe
print(data.head())

# Load images and labels
images = []
labels = []
img_dir = 'dataset'

for index, row in data.iterrows():
    img_path = os.path.join(img_dir, row['image'])
    try:
        # Open image, convert to grayscale, and resize
        img = Image.open(img_path).convert('L')
        img = img.resize((28, 28))
        
        # Convert image to numpy array and normalize
        img_array = np.array(img) / 255.0
        
        images.append(img_array)
        labels.append(row['label'])
    except FileNotFoundError:
        print(f'Warning: Image not found at {img_path}')

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Reshape images for the model (add channel dimension)
images = images.reshape(-1, 28, 28, 1)

# One-hot encode the labels
label_binarizer = LabelBinarizer()
labels = label_binarizer.fit_transform(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

print(f'Training data shape: {X_train.shape}')
print(f'Testing data shape: {X_test.shape}')

## 3. Visualize Sample Images

In [ ]:
# Get class names
class_names = label_binarizer.classes_

# Create a dictionary to store one image per class
sample_images = {class_name: None for class_name in class_names}

# Decode one-hot encoded labels for visualization
y_train_decoded = label_binarizer.inverse_transform(y_train)

for i in range(len(X_train)):
    label = y_train_decoded[i]
    if sample_images[label] is None:
        sample_images[label] = X_train[i]
    # Break if we have one sample for each class
    if all(v is not None for v in sample_images.values()):
        break

# Plot the sample images
plt.figure(figsize=(12, 8))
for i, (label, img) in enumerate(sample_images.items()):
    if img is not None:
        plt.subplot(6, 11, i + 1)
        plt.imshow(img.reshape(28, 28), cmap='gray')
        plt.title(f'Label: {label}')
        plt.axis('off')
plt.tight_layout()
plt.show()

## 4. Build the CNN Model

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

## 5. Train the Model

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.show()

## 6. Evaluate the Model

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Get predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(15, 12))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Classification Report
print('\nClassification Report:')
print(classification_report(y_true, y_pred_classes, target_names=class_names))

## 7. Save the Trained Model

In [ ]:
model.save('htr_model.h5')
print('Model saved as htr_model.h5')

## 8. Prediction Function for a New Image

In [ ]:
def predict_image(image_path, model, label_binarizer):
    try:
        # Load and preprocess the image
        img = Image.open(image_path).convert('L')
        img_resized = img.resize((28, 28))
        img_array = np.array(img_resized) / 255.0
        img_reshaped = img_array.reshape(1, 28, 28, 1)
        
        # Make a prediction
        prediction = model.predict(img_reshaped)
        predicted_class = label_binarizer.inverse_transform(prediction)[0]
        
        # Display the image and prediction
        plt.imshow(img_array, cmap='gray')
        plt.title(f'Predicted Class: {predicted_class}')
        plt.axis('off')
        plt.show()
        
        return predicted_class
    except FileNotFoundError:
        print(f'Error: Image not found at {image_path}')
        return None

# Example usage (replace with a path to a test image)
# Note: You need an image to test this. Let's use one from the test set as an example.
if len(X_test) > 0:
    # Save a test image to a temporary file to simulate a new image
    test_image_array = (X_test[0].reshape(28, 28) * 255).astype(np.uint8)
    test_image = Image.fromarray(test_image_array)
    test_image_path = 'temp_test_image.png'
    test_image.save(test_image_path)
    
    predict_image(test_image_path, model, label_binarizer)
    
    # Clean up the temporary file
    os.remove(test_image_path)